In [1]:
import csv
import json
import os
import os
import json
from pathlib import Path
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
import torch
import re

/home/dave/miniconda3/envs/unsloth_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


## Continued-Pretraining

we just concatenate all rows for each item in every column to build the pretraining set.

In [2]:
import pandas as pd
import json

# Load the CSV file
file_path = '../raw-data/simplerrawdata.csv'
df = pd.read_csv(file_path, header=None)

# Function to generate the desired JSONL format
def generate_jsonl_format(df):
    jsonl_data = []
    
    for col in range(4, df.shape[1]):  # Starting from column E (index 4)
        product = f"Simpler Trading Product: {df.iloc[3, col]}"  # E4 (index 3 - 1)
        body = (
            f"{df.iloc[3, col]}\n\n"  # E4
            f"By: \n{df.iloc[4, col]}\n\n"  # E5
            f"Available on: \n{df.iloc[5, col]}\n\n"  # E6
            f"This product is designed for: \n{df.iloc[6, col]}\n\n"  # E7
            f"{df.iloc[7, col]}\n\n"  # E8
            f"{df.iloc[11, col]}\n\n"  # E12
            f"{df.iloc[12, col]}\n\n"  # E13
            f"{df.iloc[13, col]}\n\n"  # E14
            f"{df.iloc[14, col]}\n\n"  # E15
            f"{df.iloc[15, col]}\n\n"  # E16
            f"{df.iloc[16, col]}\n\n"  # E17
            f"{df.iloc[17, col]}\n\n"  # E18
            f"{df.iloc[18, col]}\n\n"  # E19
            f"{df.iloc[19, col]}\n\n"  # E20
            f"{df.iloc[20, col]}\n\n"  # E21
            f"{df.iloc[21, col]}\n\n"  # E22
            f"{df.iloc[22, col]}\n\n"  # E23
            f"{df.iloc[23, col]}\n\n"  # E24
            f"{df.iloc[24, col]}\n\n"  # E25
            f"{df.iloc[25, col]}\n\n"  # E26
            f"{df.iloc[26, col]}\n\n"  # E27
            f"{df.iloc[27, col]}\n\n"  # E28
            f"{df.iloc[28, col]}\n\n"  # E29
            f"{df.iloc[29, col]}\n\n"  # E30
            f"{df.iloc[30, col]}\n\n"  # E31
            f"{df.iloc[31, col]}\n\n"  # E32
            f"{df.iloc[32, col]}\n\n"  # E33
            f"{df.iloc[33, col]}\n\n"  # E34
            f"{df.iloc[34, col]}\n\n"  # E35
            f"{df.iloc[35, col]}\n\n"  # E36
            f"{df.iloc[36, col]}\n\n"  # E37
            f"{df.iloc[37, col]}\n\n"  # E38
            f"{df.iloc[38, col]}\n\n"  # E39
            f"{df.iloc[3, col]} Testimonials: \n\n"  # E4
            f"{df.iloc[39, col]}\n\n"  # E40
            f"{df.iloc[40, col]}\n\n"  # E41
            f"{df.iloc[41, col]}\n\n"  # E42
            f"{df.iloc[42, col]}\n\n"  # E43
            f"{df.iloc[43, col]}\n\n"  # E44
            f"{df.iloc[44, col]}\n\n"  # E45
            f"{df.iloc[45, col]}\n\n"  # E46
            f"{df.iloc[46, col]}\n\n"  # E47
            f"{df.iloc[47, col]}\n\n"  # E48
            f"{df.iloc[48, col]}\n\n"  # E49
            f"{df.iloc[49, col]}\n\n"  # E50
            f"{df.iloc[40, col]}\n\n"  # E51
            f"{df.iloc[3, col]} Frequently Asked Questions: \n\n"  # E4
            f"{df.iloc[51, col]}\n\n"  # E52
            f"{df.iloc[52, col]}\n\n"  # E53
            f"{df.iloc[53, col]}\n\n"  # E54
            f"{df.iloc[54, col]}\n\n"  # E55
            f"{df.iloc[55, col]}\n\n"  # E56
            f"{df.iloc[56, col]}\n\n"  # E57
            f"{df.iloc[57, col]}\n\n"  # E58
            f"{df.iloc[58, col]}\n\n"  # E59
        )

        jsonl_data.append({"product": product, "body": body})
    
    return jsonl_data

# Generate the JSONL data
jsonl_data = generate_jsonl_format(df)

# Save the data to a JSONL file
output_file_path = './processed/pretrain.jsonl'
with open(output_file_path, 'w') as outfile:
    for item in jsonl_data:
        json.dump(item, outfile)
        outfile.write('\n')

# Output the path of the generated file
print(output_file_path)


./processed/pretrain.jsonl


### Clean miscellaneous nans

In [6]:

def clean_nan_in_body(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            # Parse the JSON object
            data = json.loads(line)
            
            # Clean the 'body' key if it exists
            if 'body' in data:
                body = data['body']
                # Remove "\nnan\n" first, then "nan"
                body = body.replace("\nnan\n", "").replace("nan", "")
                data['body'] = body
            
            # Write the cleaned JSON object back to the output file
            outfile.write(json.dumps(data) + '\n')
    
    print(f"Cleaned file saved as {output_file}")

# Define input and output file paths
input_file_path = './processed/pretrain.jsonl'
output_file_path = './cleaned/cleaned_pretrain.jsonl'

# Run the cleanup function
clean_nan_in_body(input_file_path, output_file_path)

# Output the path of the cleaned file
output_file_path


Cleaned file saved as ./cleaned/cleaned_pretrain.jsonl


'./cleaned/cleaned_pretrain.jsonl'

### Determine `max_seq_length` set a cutoff to save memory we use 512

In [2]:
MODEL_NAME="unsloth/gemma-2-2b-it-bnb-4bit"
max_seq_length = 512 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

==((====))==  Unsloth 2024.8: Fast Gemma2 patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 11.754 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [4]:
def split_long_sequences(text, max_length=max_seq_length):
    input_ids = tokenizer.encode_plus(text, add_special_tokens=False)["input_ids"]
    if len(input_ids) <= max_length:
        return [text]

    parts = []
    part_number = 1
    title_end_index = text.find('.  ### Article:')

    if title_end_index == -1:
        header = text[:max_length]
        content = text[max_length:]
    else:
        header = text[:title_end_index + len('.  ### Article:')]
        content = text[title_end_index + len('.  ### Article:'):]

    for i in range(0, len(input_ids), max_length):
        part_ids = input_ids[i:i + max_length]
        part_text = tokenizer.decode(part_ids, skip_special_tokens=True)

        if part_number > 1:
            part_text = f"{header} Part {part_number}.\n\n{part_text[len(header):]}"
        else:
            part_text = f"{header}{part_text[len(header):]}"

        parts.append(part_text)
        part_number += 1
    return parts

### label so the llm can make sense of it.

In [5]:
import json

input_file_path = "./cleaned/cleaned_pretrain.jsonl"
output_file_path = "split-pretrain.jsonl"

with open(input_file_path, 'r', encoding='utf-8') as infile, open(output_file_path, 'w', encoding='utf-8') as outfile:
    for line in infile:
        entry = json.loads(line)
        product = entry['product']
        body = entry['body']

        text = f"### Title: {product}.  ### Article: {body}"
        split_texts = split_long_sequences(text, max_length=max_seq_length)

        for i, split_text in enumerate(split_texts):
            if i > 0:
                split_text = split_text.replace(f"### Title: {product}.", f"### Title: {product} Part {i + 1}.")
            new_entry = {"text": split_text}
            outfile.write(json.dumps(new_entry) + '\n')


### Analyze to see what the `max_seq_lengths` are

In [6]:
def calculate_max_sequence_lengths(file_path):
    max_seq_length = 0
    total_sequences = 0
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            entry = json.loads(line)
            text = entry['text']
            tokens = tokenizer.encode_plus(text, add_special_tokens=False, max_length=None)["input_ids"]
            max_seq_length = max(max_seq_length, len(tokens))
            total_sequences += 1

    buffer = 10
    max_seq_length += buffer

    print(f"\nAnalysis for {file_path}")
    print(f"{'Measure':<20}{'Value':<14}")
    print(f"{'-'*34}")
    print(f"{'Max Seq Length':<20}{max_seq_length:<14}")
    print(f"{'Total Sequences':<20}{total_sequences:<14}")
    print(f"\nRecommendation: Set max_seq_length to at least {max_seq_length} to handle the maximum number of tokens in this dataset (Maximum + Buffer).")


In [7]:
print("\nCombined Dataset Analysis:")
calculate_max_sequence_lengths(output_file_path)


Combined Dataset Analysis:

Analysis for split-pretrain.jsonl
Measure             Value         
----------------------------------
Max Seq Length      542           
Total Sequences     184           

Recommendation: Set max_seq_length to at least 542 to handle the maximum number of tokens in this dataset (Maximum + Buffer).
